### 0. Imports and config

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import matplotlib.pyplot as plt
from pubrecon.config import Config
from pubrecon.data import DataFrame, ImagesData
from pubrecon.model import RCNN

Using TensorFlow backend.


In [23]:
# General
data_path = "../data/in/"
work_path = "../data/out/"
seed = 1337
verbose = 1

# DataFrame
dataframe_pickle_path = os.path.join(work_path, "dataframe.pickle")
force_preparation = True
subsamples = -1

# ImagesData
imagesdata_pickle_path = os.path.join(work_path, 'imagesdata.pickle')
number_of_results = 2500
iou_threshold = 0.85
max_samples = 15
show_infos = True
show_labels = True

# RCNN
model_and_weights_path = "../data/out/"
loss = None
opt = None
lr = 0.001
epochs = 1000
split_size = 0.15
checkpoint_path = os.path.join(work_path, 'checkpoint.h5')
early_stopping = True
threshold = 0.85

### 1. DataFrame

In [24]:
dataframe = DataFrame(data_path, pickle_path=dataframe_pickle_path)

In [25]:
dataframe.prepare_data(force_preparation=force_preparation, subsamples=subsamples, verbose=verbose)

In [26]:
dataframe.summary()

We have 27 classes for 180 files, total points of data is 923.


### 2. ImagesData

In [27]:
imagesdata = ImagesData(dataframe, pickle_path=imagesdata_pickle_path)

In [12]:
# That part is quite long, beware!
imagesdata.prepare_images_and_labels(number_of_results=number_of_results, iou_threshold=iou_threshold,
                                     max_samples=max_samples, verbose=verbose)

Iterating through files...: 0it [00:00, ?it/s]


In [13]:
imagesdata.summary()

We have 0 samples for 0 classes.
[]


### 3. Model

In [14]:
model = RCNN(imagesdata, loss=loss, opt=opt, lr=lr, verbose=verbose)













_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0        

/mnt/c/Users/Stache/Desktop/Cours/MA1/ML/Project/project2/pubrecon/model.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  self.model = Model(input=vggmodel.input, output=predictions)


In [ ]:
model.train(epochs=epochs, split_size=split_size, checkpoint_path=checkpoint_path, early_stopping=early_stopping,
            verbose=verbose)

### 4. Metrics

In [ ]:
loss = model.history()['loss']
val_loss = model.history()['val_loss']
accuracy = model.history()['accuracy']
val_accuracy = model.history()['val_accuracy']

In [ ]:
# Loss
plt.plot(loss)
plt.plot(val_loss)
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss", "Validation Loss"])
plt.show()

In [ ]:
print("Final loss: {}".format(loss)[-1])

In [ ]:
# Accuracy
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title("Model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy"])
plt.show()

In [ ]:
print("Final loss: {}".format(accuracy[-1]))

### 5. Recognition example

In [ ]:
imagesdata.show_image(6, show_infos=show_infos, show_labels=show_labels)

In [ ]:
model.test_image(6, show_infos=show_infos, show_labels=show_labels, number_of_results=number_of_results,
                 threshold=threshold, verbose=verbose)